# Assignment 3

Sven Den Hartog, s1003026 

Denise Klep, s4210646

In [1]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.datasets import TupleDataset
from chainer.cuda import to_cpu
from chainer import serializers
import matplotlib.pyplot as plt
import utils

In [2]:
# create toy data - compute sum of the previous and current input
def create_data(n=3000):

    X = np.random.rand(n,1).astype('float32')
    T = np.sum(np.hstack((X[0:-1],X[1:])),axis=1)
    T = np.hstack([0, T[0:]]).astype('float32')
    T = T.reshape([n,1])

    return TupleDataset(X, T)

In [17]:
class RNN(Chain):
    def __init__(self):
        super(RNN, self).__init__()
        with self.init_scope():
            self.lstm = L.LSTM(None, 20)  # the first LSTM layer
            self.out = L.Linear(None, 1)  # the feed-forward output layer

    def reset_state(self):
        self.lstm.reset_state()

    def __call__(self, x):
        h = self.lstm(x)
        y = self.out(h)
        return y

In [18]:
model = RNN()
optimizer = optimizers.SGD()
optimizer.setup(model)

In [19]:
mean_acc = [] 
mean_test_loss = [] 
mean_train_loss = []
train_losses = []
seqlen = 3000
epoch = 0
max_epoch = 20
trainset = np.array(create_data())
testset = np.array(create_data())
while epoch < max_epoch:
    model.reset_state()
    loss = 0
    count = 0
    for i,x in enumerate(trainset):
        prediction = model(np.expand_dims(x[0]-1,0))
        loss += ((prediction + 1) - (trainset[i,1,0]))**2
        count += 1
        if count % 30 == 0 or count == seqlen:
            model.cleargrads() #renew gradient calculations
            loss.backward() #runs error backpropagation
            loss.unchain_backward()
            optimizer.update() #update variables
    #print(loss.data)
    train_losses.append(loss.data)
    
    loss_test = 0
    for i,x in enumerate(testset):
        prediction = model(np.expand_dims(x[0]-1,0))
        loss_test += ((prediction + 1) - (testset[i,1,0]))**2
    print(loss_test.data)
    epoch += 1

[[ 73.98312378]]
[[ 50.34756088]]
[[ 43.77914047]]
[[ 36.10275269]]
[[ 19.8826561]]
[[ 12.75362301]]
[[ 8.7898016]]
[[ 6.8073163]]
[[ 5.65396595]]
[[ 4.95727825]]
[[ 4.50601959]]
[[ 4.19823837]]
[[ 3.97256136]]
[[ 3.79532981]]
[[ 3.64487171]]
[[ 3.50967669]]
[[ 3.38353634]]
[[ 3.26313162]]
[[ 3.1484704]]
[[ 3.03853011]]


In [ ]:
#visualize results for training and test loss values over the epochs
plt.xlabel('nr of epochs')
plt.ylabel('loss')
epochs = range(0,np.size(mean_train_loss))
plt.plot(epochs,mean_train_loss,
         label='training loss')
plt.plot(epochs,mean_test_loss,
         label='test loss')
plt.legend(loc='upper right')
plt.title('MLP MNIST classification results')
plt.show()